# Long Short-term Memory for Sentiment Classification

This notebook uses LSTM neural network on the [IMDB sentiment classification](https://keras.io/api/datasets/imdb/) task. This is a dataset for binary sentiment classification. 25,000 highly polar movie reviews are provided for training.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence


## Dataset

### Get the data
IMDB sentiment dataset is available in keras.datasets.

In [2]:
help(imdb.load_data)

Help on function load_data in module tensorflow.python.keras.datasets.imdb:

load_data(path='imdb.npz', num_words=None, skip_top=0, maxlen=None, seed=113, start_char=1, oov_char=2, index_from=3, **kwargs)
    Loads the [IMDB dataset](https://ai.stanford.edu/~amaas/data/sentiment/).
    
    This is a dataset of 25,000 movies reviews from IMDB, labeled by sentiment
    (positive/negative). Reviews have been preprocessed, and each review is
    encoded as a list of word indexes (integers).
    For convenience, words are indexed by overall frequency in the dataset,
    so that for instance the integer "3" encodes the 3rd most frequent word in
    the data. This allows for quick filtering operations such as:
    "only consider the top 10,000 most
    common words, but eliminate the top 20 most common words".
    
    As a convention, "0" does not stand for a specific word, but instead is used
    to encode any unknown word.
    
    Args:
      path: where to cache the data (relative to `~

In [3]:
max_features = 20000
maxlen = 80
# maxlen: cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...


<__array_function__ internals>:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:155: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])


25000 train sequences
25000 test sequences


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/tensorflow/python/keras/datasets/imdb.py:156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


### Data Preprocessing

Keras has already preprocessed the data

In [4]:
print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
X_train[1]

Pad sequences (samples x time)
X_train shape: (25000, 80)
X_test shape: (25000, 80)


array([ 125,   68,    2, 6853,   15,  349,  165, 4362,   98,    5,    4,
        228,    9,   43,    2, 1157,   15,  299,  120,    5,  120,  174,
         11,  220,  175,  136,   50,    9, 4373,  228, 8255,    5,    2,
        656,  245, 2350,    5,    4, 9837,  131,  152,  491,   18,    2,
         32, 7464, 1212,   14,    9,    6,  371,   78,   22,  625,   64,
       1382,    9,    8,  168,  145,   23,    4, 1690,   15,   16,    4,
       1355,    5,   28,    6,   52,  154,  462,   33,   89,   78,  285,
         16,  145,   95], dtype=int32)

## RNN

### Build the RNN model

In [5]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.SimpleRNN(32, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          320000    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                1568      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 321,601
Trainable params: 321,601
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [7]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 6s 15ms/step - loss: 0.7013 - accuracy: 0.5005 - val_loss: 0.6872 - val_accuracy: 0.5424
Epoch 2/32
313/313 [==============================] - 4s 14ms/step - loss: 0.6731 - accuracy: 0.5748 - val_loss: 0.5835 - val_accuracy: 0.6996
Epoch 3/32
313/313 [==============================] - 4s 14ms/step - loss: 0.5444 - accuracy: 0.7274 - val_loss: 0.5230 - val_accuracy: 0.7602
Epoch 4/32
313/313 [==============================] - 5s 15ms/step - loss: 0.4429 - accuracy: 0.8026 - val_loss: 0.9065 - val_accuracy: 0.7028
Epoch 5/32
313/313 [==============================] - 5s 15ms/step - loss: 0.3964 - accuracy: 0.8294 - val_loss: 0.4848 - val_accuracy: 0.7810
Epoch 6/32
313/313 [==============================] - 5s 16ms/step - loss: 0.3652 - accuracy: 0.8479 - val_loss: 0.4059 - val_accuracy: 0.8176
Epoch 7/32
313/313 [==============================] - 5s 16ms/step - loss: 0.3368 - accuracy: 0.8644 - val_loss: 0.5097 - val_accuracy: 0.8106

In [8]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 78.16%


## RNN using the entire sequence instead of the last output

In [9]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16, input_length=maxlen))
model.add(layers.SimpleRNN(32, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 80, 16)            320000    
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 80, 32)            1568      
_________________________________________________________________
flatten (Flatten)            (None, 2560)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2561      
Total params: 324,129
Trainable params: 324,129
Non-trainable params: 0
_________________________________________________________________


In [10]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 6s 15ms/step - loss: 0.7005 - accuracy: 0.5197 - val_loss: 0.6337 - val_accuracy: 0.6820
Epoch 2/32
313/313 [==============================] - 5s 16ms/step - loss: 0.4859 - accuracy: 0.7594 - val_loss: 0.3670 - val_accuracy: 0.8348
Epoch 3/32
313/313 [==============================] - 5s 15ms/step - loss: 0.3209 - accuracy: 0.8605 - val_loss: 0.3637 - val_accuracy: 0.8434
Epoch 4/32
313/313 [==============================] - 5s 15ms/step - loss: 0.2655 - accuracy: 0.8905 - val_loss: 0.3518 - val_accuracy: 0.8450
Epoch 5/32
313/313 [==============================] - 5s 15ms/step - loss: 0.2337 - accuracy: 0.9050 - val_loss: 0.3687 - val_accuracy: 0.8476
Epoch 6/32
313/313 [==============================] - 5s 15ms/step - loss: 0.2045 - accuracy: 0.9199 - val_loss: 0.3834 - val_accuracy: 0.8420


In [11]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.01%


## LSTM

### Build the model

In [12]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

### Inspect the model

Use the `.summary` method to print a simple description of the model

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               74240     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 394,369
Trainable params: 394,369
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [14]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 23s 65ms/step - loss: 0.5300 - accuracy: 0.7449 - val_loss: 0.4115 - val_accuracy: 0.8256
Epoch 2/32
313/313 [==============================] - 19s 61ms/step - loss: 0.3369 - accuracy: 0.8603 - val_loss: 0.3569 - val_accuracy: 0.8456
Epoch 3/32
313/313 [==============================] - 20s 64ms/step - loss: 0.2840 - accuracy: 0.8844 - val_loss: 0.3470 - val_accuracy: 0.8490
Epoch 4/32
313/313 [==============================] - 20s 64ms/step - loss: 0.2483 - accuracy: 0.9015 - val_loss: 0.3547 - val_accuracy: 0.8474
Epoch 5/32
313/313 [==============================] - 19s 60ms/step - loss: 0.2303 - accuracy: 0.9129 - val_loss: 0.3786 - val_accuracy: 0.8454


In [15]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.91%


## Stacked LSTM

### Build the model

In [16]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 16)          320000    
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 128)         74240     
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 525,953
Trainable params: 525,953
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [17]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

Epoch 1/32
313/313 [==============================] - 45s 132ms/step - loss: 0.5047 - accuracy: 0.7484 - val_loss: 0.4167 - val_accuracy: 0.8234
Epoch 2/32
313/313 [==============================] - 44s 139ms/step - loss: 0.3336 - accuracy: 0.8612 - val_loss: 0.5216 - val_accuracy: 0.8244
Epoch 3/32
313/313 [==============================] - 43s 138ms/step - loss: 0.2802 - accuracy: 0.8855 - val_loss: 0.3630 - val_accuracy: 0.8412
Epoch 4/32
313/313 [==============================] - 45s 144ms/step - loss: 0.2499 - accuracy: 0.9029 - val_loss: 0.5825 - val_accuracy: 0.8138
Epoch 5/32
313/313 [==============================] - 43s 138ms/step - loss: 0.2296 - accuracy: 0.9099 - val_loss: 0.3626 - val_accuracy: 0.8428
Epoch 6/32
313/313 [==============================] - 46s 147ms/step - loss: 0.2098 - accuracy: 0.9198 - val_loss: 0.5436 - val_accuracy: 0.8294
Epoch 7/32
313/313 [==============================] - 45s 144ms/step - loss: 0.1998 - accuracy: 0.9255 - val_loss: 0.3799 - val_ac

In [18]:
_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Testing set accuracy: 83.40%


## Bidirectional LSTM

In [6]:
model = keras.Sequential()
# Embedding layer turns vectors of integers into dense real vectors of fixed size
model.add(layers.Embedding(max_features, 16))
model.add(layers.Bidirectional(layers.LSTM(128, return_sequences=False, dropout=0.2, recurrent_dropout=0.2)))
model.add(layers.Dense(1, activation='sigmoid'))

optimizer = optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          320000    
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               148480    
_________________________________________________________________
dense (Dense)                (None, 1)                 257       
Total params: 468,737
Trainable params: 468,737
Non-trainable params: 0
_________________________________________________________________


### Train the model

In [7]:
EPOCHS = 32
BATCH = 64

early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)

model.fit(X_train, y_train,
          batch_size=BATCH,
          epochs=EPOCHS,
          validation_split=0.2,
          verbose = 1,
          callbacks = [early_stop])

_, acc = model.evaluate(X_test, y_test, batch_size=64, verbose = 0)
print("Testing set accuracy: {:.2f}%".format(acc*100))

Epoch 1/32
313/313 [==============================] - 34s 96ms/step - loss: 0.5161 - accuracy: 0.7369 - val_loss: 0.3840 - val_accuracy: 0.8386
Epoch 2/32
313/313 [==============================] - 29s 93ms/step - loss: 0.3292 - accuracy: 0.8626 - val_loss: 0.3500 - val_accuracy: 0.8500
Epoch 3/32
313/313 [==============================] - 29s 94ms/step - loss: 0.2698 - accuracy: 0.8923 - val_loss: 0.3617 - val_accuracy: 0.8486
Epoch 4/32
313/313 [==============================] - 30s 96ms/step - loss: 0.2350 - accuracy: 0.9106 - val_loss: 0.3503 - val_accuracy: 0.8460
Testing set accuracy: 83.98%
